# Data Processing Pipeline

In [4]:
import importlib
import preprocess_data
import regularize_tracks
import trajectory_segmentation
importlib.reload(preprocess_data)
importlib.reload(regularize_tracks)
importlib.reload(trajectory_segmentation)
from regularize_tracks import resample_segment

from preprocess_data import process_zip, drop_duplicate_messages, save_parquet_partitioned
from trajectory_segmentation import segment_trajectories
from download_data import download_ais_range

import matplotlib.pyplot as plt
import os
import glob
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


## Input Parameters

In [7]:
START_DATE =  datetime(2025, 8, 1) # None to skip downloading
END_DATE  = datetime(2025, 8, 2) # None to skip downloading

# Directory where to store ZIP files
ZIP_DIR = os.path.join("..", "..", "data", "aisdk", "raw")
# Where to store the Parquet dataset
PARQUET_DIR = os.path.join("..", "..", "data", "aisdk", "interim", "aisdk_2025")

# Final Parquet file path
PARQUET_FILE_FINAL = os.path.join("..", "..", "data", "aisdk", "processed", "aisdk_2025")

## Data Download

In [8]:
if START_DATE and END_DATE:
    download_ais_range(start_date = START_DATE,
                       end_date = END_DATE,
                       output_dir = ZIP_DIR)

Date range: 2025-08-01 → 2025-08-02 (exclusive)

→ 2025-08-01: http://aisdata.ais.dk/aisdk-2025-08-01.zip
   Saved: ../../data/aisdk/raw/aisdk-2025-08-01.zip

Done!


## Data Preprocessing

In [97]:
os.makedirs(PARQUET_DIR, exist_ok=True)

# Process all August 2025 ZIP files
pattern = os.path.join(ZIP_DIR, "aisdk-2025-08-*.zip")
zip_files = sorted(glob.glob(pattern))

print(f"Found {len(zip_files)} ZIP files to process.")

for zp in zip_files:
    process_zip(zp, PARQUET_DIR)

print("All files processed.")

Found 1 ZIP files to process.

=== Processing ../../data/aisdk/raw/aisdk-2025-08-01.zip ===
Output path: ../../data/aisdk/interim/aisdk_2025
Reading ../../data/aisdk/raw/aisdk-2025-08-01.zip ...
Columns in raw DF: ['# Timestamp', 'Type of mobile', 'MMSI', 'Latitude', 'Longitude', 'SOG', 'COG', 'Ship type']
Filtered to Ship type == Cargo and dropped column.
Applied geographic bounding box (60, 0, 50, 20).
Filtered to Type of mobile in ['Class A', 'Class B'] and dropped column.
Applied MMSI format and MID filters.
Parsed Timestamp column.
Converted SOG from knots to m/s.
Final columns: ['Timestamp', 'MMSI', 'Latitude', 'Longitude', 'SOG', 'COG', 'UTM_x', 'UTM_y', 'UTM_zone', 'UTM_letter']
Rows after filtering: 3470997
Saving to parquet dataset at ../../data/aisdk/interim/aisdk_2025 ...
Parquet save done.
=== Done for ../../data/aisdk/raw/aisdk-2025-08-01.zip ===

All files processed.


In [98]:
df = pd.read_parquet(PARQUET_DIR)

In [100]:
df.head()

,Timestamp,Latitude,Longitude,SOG,COG,UTM_x,UTM_y,UTM_zone,UTM_letter,MMSI
0,2025-08-01 00:02:24,57.112585,12.245682,0.0,167.7,333212.967255,6.333286e+06,33,V,205136000
1,2025-08-01 00:08:20,57.112590,12.245678,0.0,17.2,333212.747587,6.333286e+06,33,V,205136000
2,2025-08-01 00:08:24,57.112590,12.245678,0.0,17.2,333212.747587,6.333286e+06,33,V,205136000
3,2025-08-01 00:14:21,57.112583,12.245682,0.0,135.0,333212.958265,6.333286e+06,33,V,205136000
4,2025-08-01 00:14:24,57.112583,12.245682,0.0,135.0,333212.958265,6.333286e+06,33,V,205136000


## Create trajectories

In [101]:
df = drop_duplicate_messages(df)

df = segment_trajectories(df, 
                              sog_threshold=0.5, # 1 knot in m/s
                             position_threshold=50, # 50 meters
                              time_threshold=30 # 30 minutes
                             )

Dropped 6409610 duplicate (Timestamp, MMSI) rows.
Processing ship 1
Processing ship 2
Processing ship 3
Processing ship 4
Processing ship 5
Processing ship 6
Processing ship 7
Processing ship 8
Processing ship 9
Processing ship 10
Processing ship 11
Processing ship 12
Processing ship 13
Processing ship 14
Processing ship 15
Processing ship 16
Processing ship 17
Processing ship 18
Processing ship 19
Processing ship 20
Processing ship 21
Processing ship 22
Processing ship 23
Processing ship 24
Processing ship 25
Processing ship 26
Processing ship 27
Processing ship 28
Processing ship 29
Processing ship 30
Processing ship 31
Processing ship 32
Processing ship 33
Processing ship 34
Processing ship 35
Processing ship 36
Processing ship 37
Processing ship 38
Processing ship 39
Processing ship 40
Processing ship 41
Processing ship 42
Processing ship 43
Processing ship 44
Processing ship 45
Processing ship 46
Processing ship 47
Processing ship 48
Processing ship 49
Processing ship 50
Processin

## Missing values

In [102]:
# Missing segments 
mask_missing = df["SOG"].isna() | df["COG"] .isna()
bad_segments = df[mask_missing][["MMSI", "Trajectory"]].drop_duplicates()

# All unique segments
all_segments = df[["MMSI", "Trajectory"]].drop_duplicates()

# Clean segments = all_segments MINUS bad_segments
clean_segments = all_segments.merge(
    bad_segments,
    on=["MMSI", "Trajectory"],
    how="left",
    indicator=True
).query('_merge == "left_only"').drop(columns="_merge")

print(clean_segments)

clean_df = df.merge(clean_segments, on=["MMSI", "Trajectory"], how="inner")

          MMSI  Trajectory
0    209014000           0
1    209184000           1
2    209276000           2
3    209314000           3
4    209336000           4
..         ...         ...
529  636025570         529
530  636091784         530
531  636092635         531
532  636093117         532
533  636093288         533

[514 rows x 2 columns]


In [117]:
# Check how many segments were removed
print("Number of segments before cleaning:", all_segments.shape)
print("Number of segments after cleaning:", clean_segments.shape)
print("Number of removed segments:", bad_segments.shape)

Number of segments before cleaning: (534, 2)
Number of segments after cleaning: (514, 2)
Number of removed segments: (20, 2)


## Resampling and Imputation

In [71]:
df_resampled = clean_df.groupby(["MMSI", "Trajectory"]).apply(resample_segment)
df_resampled = df_resampled.reset_index(names=["MMSI", "Trajectory", "Timestamp"])

/var/folders/7z/72trz_6j2ss4z5yp7pd0jr_00000gn/T/ipykernel_56506/4250933535.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_resampled = clean_df.groupby(["MMSI", "Trajectory"]).apply(resample_segment)
/var/folders/7z/72trz_6j2ss4z5yp7pd0jr_00000gn/T/ipykernel_56506/4250933535.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_resampled = clean_df.groupby(["MMSI", "Trajectory"]).apply(resample_segment)


In [111]:
df_resampled.head()

,MMSI,Trajectory,Timestamp,COG,UTM_x,UTM_y,SOG
0,209014000,0,2025-08-01 19:37:13,254.700000,612411.419016,6.543974e+06,4.321330
1,209014000,0,2025-08-01 19:38:13,255.900012,612156.960477,6.543900e+06,4.471913
2,209014000,0,2025-08-01 19:39:13,258.164931,611900.143260,6.543831e+06,4.493230
3,209014000,0,2025-08-01 19:40:13,262.075074,611624.337855,6.543777e+06,4.578552
4,209014000,0,2025-08-01 19:41:13,265.452968,611354.089283,6.543744e+06,4.611012


In [73]:
# Quick check for missing values
df_resampled.isnull().sum()

MMSI          0
Trajectory    0
Timestamp     0
COG           0
UTM_x         0
UTM_y         0
SOG           0
dtype: int64

## Degrees conversion

In [107]:
# Decompose COG into its vector components
cog_radians = np.radians(df_resampled['COG']) # convert to radians
df_resampled['v_east'] = df_resampled['SOG'] * np.sin(cog_radians) # eastward component
df_resampled['v_north'] = df_resampled['SOG'] * np.cos(cog_radians) # northward component
df_resampled.drop(columns=['COG'], inplace=True)

In [108]:
df_resampled.head()

,MMSI,Trajectory,Timestamp,UTM_x,UTM_y,SOG,v_east,v_north
0,209014000,0,2025-08-01 19:37:13,612411.419016,6.543974e+06,4.321330,-4.168171,-1.140282
1,209014000,0,2025-08-01 19:38:13,612156.960477,6.543900e+06,4.471913,-4.337184,-1.089424
2,209014000,0,2025-08-01 19:39:13,611900.143260,6.543831e+06,4.493230,-4.397713,-0.921540
3,209014000,0,2025-08-01 19:40:13,611624.337855,6.543777e+06,4.578552,-4.534824,-0.631270
4,209014000,0,2025-08-01 19:41:13,611354.089283,6.543744e+06,4.611012,-4.596499,-0.365549


In [109]:
df_resampled.shape

(270778, 8)

## Write to a final parquet file

In [77]:
save_parquet_partitioned(df_resampled, out_path=PARQUET_FILE_FINAL, partition_cols=["MMSI", "Trajectory"])

Saving to parquet dataset at ../../data/aisdk/processed/aisdk_2025 ...
Parquet save done.
